## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from API_key import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
path = 'C:/Users/Deva/Desktop/Bootcamp/Repos/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv'
city_data_df = pd.read_csv(path)
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.10,-79.24,54.00,70,40,10.29,scattered clouds
1,1,Khatanga,RU,71.97,102.50,22.55,97,100,17.60,overcast clouds
2,2,Vaini,TO,-21.20,-175.20,71.60,100,90,12.75,light intensity shower rain
3,3,Atuona,PF,-9.80,-139.03,77.05,75,0,19.73,clear sky
4,4,Bluff,NZ,-46.60,168.33,44.01,97,89,8.72,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Atuona,PF,-9.80,-139.03,77.05,75,0,19.73,clear sky
6,6,Celestun,MX,20.87,-90.40,79.59,84,100,24.70,moderate rain
9,9,Travnik,BA,44.23,17.67,77.00,31,0,8.05,clear sky
13,13,Kapaa,US,22.08,-159.32,80.60,74,75,9.17,broken clouds
19,19,Maputo,MZ,-25.97,32.59,77.00,78,20,11.41,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database.csv    228
City                      228
Country                   228
Lat                       228
Lng                       228
Max Temp                  228
Humidity                  228
Cloudiness                228
Wind Speed                228
Current Description       228
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy() 

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Atuona,PF,77.05,clear sky,-9.80,-139.03,
6,Celestun,MX,79.59,moderate rain,20.87,-90.40,
9,Travnik,BA,77.00,clear sky,44.23,17.67,
13,Kapaa,US,80.60,broken clouds,22.08,-159.32,
19,Maputo,MZ,77.00,few clouds,-25.97,32.59,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="Vacation_Search")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Recommended Hotel</dt><dd>{Hotel Name}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
print(len(locations))

228


In [17]:
# 11a. Add a marker layer for each city to the map.
# Get the maximum temperature.
max_temp = clean_hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=temps,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))